In [1]:
import numpy as np

In [6]:
a = np.identity(3)

In [12]:
a * b

array([[ 1.,  0.,  0.],
       [ 0.,  2.,  0.],
       [ 0.,  0.,  1.]])

In [11]:
b = np.vstack((np.array([1,2]).reshape(-1,1),1))